In [19]:
!pip install nltk
import nltk
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')


Defaulting to user installation because normal site-packages is not writeable


[nltk_data] Downloading package stopwords to
[nltk_data]     /home/u190532/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /home/u190532/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /home/u190532/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [20]:
import pandas as pd
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import string

# Load dataset
df = pd.read_csv("./training_data.csv")

# Remove unnecessary columns
df = df.drop(['Source', 'PublishDate'], axis=1)

# Remove missing values
df = df.dropna()

# Define stopwords and lemmatizer
stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()

# Define function for text preprocessing
def preprocess_text(text):
    # Convert to lowercase
    text = text.lower()
    
    # Remove punctuation
    text = text.translate(str.maketrans("", "", string.punctuation))
    
    # Tokenize text
    tokens = nltk.word_tokenize(text)
    
    # Remove stopwords
    tokens = [token for token in tokens if token not in stop_words]
    
    # Lemmatize tokens
    tokens = [lemmatizer.lemmatize(token) for token in tokens]
    
    # Join tokens to form text again
    text = " ".join(tokens)
    
    return text

# Preprocess text in the dataset
df['clean_title'] = df['Title'].apply(preprocess_text)


In [21]:
df['clean_headline'] = df['Headline'].apply(preprocess_text)

In [4]:
!pip install gensim
import gensim

# Convert preprocessed text to gensim dictionary and corpus
texts = [doc.split() for doc in df['clean_title']]
dictionary = gensim.corpora.Dictionary(texts)
corpus = [dictionary.doc2bow(text) for text in texts]

# Define number of topics and run LDA model
num_topics = 4
lda_model = gensim.models.LdaModel(corpus, num_topics=num_topics, id2word=dictionary, passes=10)

# Print top words in each topic
for topic in lda_model.show_topics(num_topics=num_topics):
    print(topic)


Defaulting to user installation because normal site-packages is not writeable
  Using cached gensim-4.3.1-cp39-cp39-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (26.5 MB)
(0, '0.059*"economy" + 0.023*"china" + 0.014*"new" + 0.011*"budget" + 0.008*"’" + 0.008*"show" + 0.007*"sander" + 0.006*"pick" + 0.006*"market" + 0.006*"economic"')
(1, '0.101*"microsoft" + 0.016*"window" + 0.016*"microsofts" + 0.014*"10" + 0.012*"new" + 0.011*"surface" + 0.008*"one" + 0.007*"make" + 0.007*"cloud" + 0.007*"lumia"')
(2, '0.130*"obama" + 0.030*"obamas" + 0.023*"palestine" + 0.020*"president" + 0.013*"trump" + 0.012*"palestinian" + 0.010*"court" + 0.009*"state" + 0.009*"say" + 0.007*"visit"')
(3, '0.097*"economy" + 0.016*"u" + 0.012*"2016" + 0.012*"economic" + 0.011*"say" + 0.011*"global" + 0.010*"growth" + 0.008*"year" + 0.007*"oil" + 0.006*"gun"')


In [5]:
import gensim

# Convert preprocessed text to gensim dictionary and corpus
texts = [doc.split() for doc in df['clean_headline']]
dictionary = gensim.corpora.Dictionary(texts)
corpus = [dictionary.doc2bow(text) for text in texts]

# Define number of topics and run LDA model
num_topics = 5
lda_model = gensim.models.LdaModel(corpus, num_topics=num_topics, id2word=dictionary, passes=10)

# Print top words in each topic
for topic in lda_model.show_topics(num_topics=num_topics):
    print(topic)

(0, '0.015*"microsoft" + 0.010*"new" + 0.009*"company" + 0.008*"business" + 0.007*"cloud" + 0.006*"palestine" + 0.006*"service" + 0.005*"research" + 0.004*"technology" + 0.004*"help"')
(1, '0.053*"microsoft" + 0.014*"new" + 0.014*"window" + 0.010*"10" + 0.008*"microsofts" + 0.008*"surface" + 0.008*"’" + 0.007*"company" + 0.007*"one" + 0.005*"user"')
(2, '0.052*"economy" + 0.023*"economic" + 0.011*"global" + 0.011*"said" + 0.010*"market" + 0.010*"china" + 0.009*"u" + 0.009*"minister" + 0.008*"world" + 0.007*"government"')
(3, '0.027*"economy" + 0.022*"year" + 0.014*"percent" + 0.012*"2015" + 0.010*"quarter" + 0.010*"last" + 0.009*"month" + 0.008*"u" + 0.008*"growth" + 0.006*"palestine"')
(4, '0.063*"obama" + 0.057*"president" + 0.033*"barack" + 0.010*"state" + 0.010*"said" + 0.010*"obamas" + 0.010*"u" + 0.009*"palestinian" + 0.009*"republican" + 0.008*"house"')


In [29]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import LinearSVR
from sklearn.metrics import classification_report

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(df['clean_title'], df['SentimentTitle'], test_size=0.2, random_state=42)

# Vectorize text using TF-IDF
vectorizer = TfidfVectorizer()
X_train_vec = vectorizer.fit_transform(X_train)
X_test_vec = vectorizer.transform(X_test)

print(X_train_vec.shape)
print(X_test_vec.shape)
x_title = X_train

(32406, 18481)
(8102, 18481)


In [23]:
print(y_train.value_counts())


 0.000000    6645
 0.041667     409
 0.044194     368
-0.041667     299
 0.039528     283
             ... 
-0.036182       1
 0.027862       1
-0.091213       1
 0.073546       1
-0.173529       1
Name: SentimentTitle, Length: 6601, dtype: int64


In [30]:
# Train SVM classifier
svmTitle = LinearSVR()

svmTitle.fit(X_train_vec, y_train)

# Predict on test data
y_pred = svmTitle.predict(X_test_vec)

/opt/intel/inteloneapi/intelpython/latest/lib/python3.9/site-packages/sklearn/svm/_base.py:1225: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


In [31]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import LinearSVC
from sklearn.metrics import classification_report

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(df['clean_headline'], df['SentimentHeadline'], test_size=0.2, random_state=42)

# Vectorize text using TF-IDF
vectorizer = TfidfVectorizer()
X_train_vec = vectorizer.fit_transform(X_train)
X_test_vec = vectorizer.transform(X_test)
print(X_train_vec.shape)
print(X_test_vec.shape)
x_Headline = X_train

(32406, 33111)
(8102, 33111)


In [32]:
# Train SVM classifier
svmHeadline = LinearSVR()
svmHeadline.fit(X_train_vec, y_train)

# Predict on test data
y_pred = svmHeadline.predict(X_test_vec)

/opt/intel/inteloneapi/intelpython/latest/lib/python3.9/site-packages/sklearn/svm/_base.py:1225: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


In [33]:
# Load dataset
dfTest = pd.read_csv("./test_data.csv")

In [34]:
dfTest['clean_title'] = dfTest['Title'].apply(preprocess_text)
dfTest['clean_headline'] = dfTest['Headline'].apply(preprocess_text)

In [35]:
vectorizer = TfidfVectorizer()
X_train_vec = vectorizer.fit_transform(x_title)
X_test_vec = vectorizer.transform(dfTest['clean_title'])
print(X_train_vec.shape)
print(X_test_vec.shape)
y_predTitle = svmTitle.predict(X_test_vec)
print(y_predTitle)

(32406, 18481)
(15424, 18481)
[-0.00225646  0.07821524 -0.019051   ...  0.00142398  0.09807077
  0.05414269]


In [36]:
vectorizer = TfidfVectorizer()
X_train_vec = vectorizer.fit_transform(x_Headline)
X_test_vec = vectorizer.transform(dfTest['clean_headline'])

y_predHeadline = svmHeadline.predict(X_test_vec)
print(y_predHeadline)

[-0.00776357  0.1488848  -0.23006005 ... -0.02424033 -0.05251094
  0.17786623]


In [37]:
print(type(y_predTitle))
print(type(y_predHeadline))

<class 'numpy.ndarray'>
<class 'numpy.ndarray'>


In [38]:
y_idLink = dfTest['IDLink'].to_numpy()
print(type(y_idLink))

<class 'numpy.ndarray'>


In [ ]:
submission = pd.DataFrame({
    "IDLink": y_idLink,
    "SentimentTitle": y_predTitle,
    "SentimentHeadline": y_predHeadline
})
submission.to_csv("submission.csv", index=False)